Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

# Make a node

In [2]:
μ = rand(5, 5)
σ = rand(5, 5)
aa = RandomNode(:a, [:i, :j], Normal, μ, σ)
bb = RandomNode(:b, [:j, :k], Gamma, rand(5, 3), rand(5, 3))

VB.RandomNode{Distributions.Gamma}(:b,[:j,:k],5x3 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=0.35605574756886527, θ=0.590984531049549)     …  Distributions.Gamma(α=0.16592550267966266, θ=0.21431415469287596) 
 Distributions.Gamma(α=0.0071272850750627725, θ=0.5798013693671358)     Distributions.Gamma(α=0.5381103291721503, θ=0.6348784094066386)   
 Distributions.Gamma(α=0.8985568845705709, θ=0.7009424593727227)        Distributions.Gamma(α=0.20371731655127134, θ=0.025895262513009776)
 Distributions.Gamma(α=0.8106878794285928, θ=0.9268960474104413)        Distributions.Gamma(α=0.7898292144967123, θ=0.7310776997085937)   
 Distributions.Gamma(α=0.5234555657100284, θ=0.9057207948205108)        Distributions.Gamma(α=0.6746708411267228, θ=0.30636178554996496)  )

In [3]:
cc = ConstantNode(:c, [:i, :j], rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.124463  0.922355   0.448002  0.12264 
 0.209865  0.338072   0.230992  0.827531
 0.142302  0.0639229  0.993829  0.565913)

In [4]:
cc = ConstantNode(rand(3, 4), [:i, :j])

VB.ConstantNode{Float64}(symbol("##const#7735"),[:i,:j],3x4 Array{Float64,2}:
 0.196672  0.872916  0.0743068  0.834964
 0.967775  0.767241  0.764598   0.757284
 0.327708  0.595233  0.78208    0.950987)

In [5]:
nodes = Node[aa, bb]
fi = get_structure(nodes...)
fi

VB.FactorInds([:i,:j,:k],[5,3,5],Dict(:a=>[1,2],:b=>[2,3]))

In [6]:
a[i, j] ~ Normal(μ, σ)

VB.RandomNode{Distributions.Normal}(:a,[:i,:j],5x5 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.7629787660085989, σ=0.9496204636564747)  …  Distributions.Normal(μ=0.5545830017150288, σ=0.4819055725258383) 
 Distributions.Normal(μ=0.7121983592383938, σ=0.77164509267781)       Distributions.Normal(μ=0.5479000134455572, σ=0.3954419462583958) 
 Distributions.Normal(μ=0.3468491482199798, σ=0.7769480362139563)     Distributions.Normal(μ=0.7961740035981721, σ=0.37067372152918376)
 Distributions.Normal(μ=0.7999399512970777, σ=0.5045433569747941)     Distributions.Normal(μ=0.8080869186596182, σ=0.42903420516168556)
 Distributions.Normal(μ=0.9176232497470909, σ=0.3900040025843241)     Distributions.Normal(μ=0.5173321545429217, σ=0.25237028619669766))

In [7]:
c[i, j] ~ Const(rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.550413  0.66825   0.0247327  0.682582 
 0.343088  0.239031  0.462462   0.0698694
 0.973105  0.221424  0.827344   0.886473 )

# Make a factor

In [8]:
dims = (10, 2)

μ[j] ~ Normal(rand(dims[2]), ones(dims[2]))
τ ~ Gamma(1, 1)
x[i, j] ~ Normal(rand(dims), ones(dims))

f = @factor LogNormalFactor x μ τ;

# Make a (generated) value function

In [9]:
value(f)

-34.61227021312123

In [10]:
g = @factor EntropyFactor x
value(g)

28.378770664093462

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [11]:
dims = (20, 6)

# note: it won't matter much how we initialize here
μ[j] ~ Normal(zeros(dims[2]), ones(dims[2]))
τ[j] ~ Gamma(1.1 * ones(dims[2]), ones(dims[2]))
μ0[j] ~ Const(zeros(dims[2]))
τ0[j] ~ Const(2 * ones(dims[2]))
a0[j] ~ Const(1.1 * ones(dims[2]))
b0[j] ~ Const(ones(dims[2]))

y[i, j] ~ Const(rand(dims));

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [12]:
obs = @factor LogNormalFactor y μ τ
μ_prior = @factor LogNormalFactor μ μ0 τ0
τ_prior = @factor LogGammaFactor τ a0 b0

VB.LogGammaFactor{1}(VB.RandomNode{Distributions.Gamma}(:τ,[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:a0,[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.FactorInds([:j],[6],Dict(:τ=>[1],:b0=>[1],:a0=>[1])),Dict(:τ=>:x,:b0=>:β,:a0=>:α))

In [13]:
value(obs), value(μ_prior), value(τ_prior)

(-171.26136081974116,-13.593072740907873,0.0449816833123928)

In [14]:
m = VBModel([μ, τ, μ0, τ0, a0, b0, y], [obs, μ_prior, τ_prior])

VB.VBModel(VB.Node[VB.RandomNode{Distributions.Normal}(:μ,[:j],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)]),VB.RandomNode{Distributions.Gamma}(:τ,[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:μ0,[:j],[0.0,0.0,0.0,0.0,0.0,0.0]),VB.ConstantNode{Float64}(:τ0,[:j],[2.0,2.0,2.0,2.0,2.0,2.0]),VB.ConstantNode{Float64}(:a0,[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.ConstantNode{Float64}(:y,[:i,:j],20x6 Array{Float64,2}:
 0.0072136  0.251249   0.809277   0.988777   0.137172  0.711034 
 0.642755   0.393941   0.31775    0.311519   0.39197   0.434079 
 0.156588   0.625304   0.401468   0.65933

In [15]:
[n.name for n in m.nodes]

7-element Array{Any,1}:
 :μ 
 :τ 
 :μ0
 :τ0
 :a0
 :b0
 :y 

In [16]:
[typeof(f) for f in m.factors]

3-element Array{DataType,1}:
 VB.LogNormalFactor{2}
 VB.LogNormalFactor{1}
 VB.LogGammaFactor{1} 

In [17]:
m.graph

Dict{VB.Node,Array{Tuple{VB.Factor{N},Symbol},1}} with 7 entries:
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…

In [18]:
naturals(obs, μ)

6-element Array{Tuple{Float64,Float64},1}:
 (9.8402860670654,-11.000000000000002)   
 (10.14927041517417,-11.000000000000002) 
 (11.44693078084976,-11.000000000000002) 
 (11.517143112785575,-11.000000000000002)
 (10.689238169860024,-11.000000000000002)
 (8.642564743319356,-11.000000000000002) 

In [19]:
naturals(obs, τ)

6-element Array{Tuple{Float64,Float64},1}:
 (10.0,12.986328066415775)
 (10.0,12.952426900577697)
 (10.0,13.666352185132364)
 (10.0,13.665455336730794)
 (10.0,13.151192067085534)
 (10.0,12.13645749846231) 

In [20]:
check_conjugate(τ, m)

true

In [21]:
check_conjugate(μ, m)

true

In [22]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)

In [23]:
τ.data

6-element Array{Distributions.Gamma,1}:
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)

In [24]:
update!(m)

In [25]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=-0.8200415862145785, σ=0.7071067811865476) 
 Distributions.Normal(μ=-0.6015566583863617, σ=0.7071067811865476) 
 Distributions.Normal(μ=0.31602778585986285, σ=0.7071067811865476) 
 Distributions.Normal(μ=0.36567540189459874, σ=0.7071067811865476) 
 Distributions.Normal(μ=-0.21974179742592012, σ=0.7071067811865476)
 Distributions.Normal(μ=-1.6669584562071338, σ=0.7071067811865476) 

In [26]:
τ.data

6-element Array{Distributions.Gamma,1}:
 Distributions.Gamma(α=33.046868093303104, θ=0.8999999999999999)
 Distributions.Gamma(α=28.121459393078027, θ=0.8999999999999999)
 Distributions.Gamma(α=17.376407627157185, θ=0.8999999999999999)
 Distributions.Gamma(α=17.17397129910631, θ=0.8999999999999999) 
 Distributions.Gamma(α=21.76939519568054, θ=0.8999999999999999) 
 Distributions.Gamma(α=59.02105006584324, θ=0.8999999999999999) 